In [1]:
!pip install nltk

import re
from collections import defaultdict

import nltk
from collections import Counter

from operator import itemgetter

import numpy as np

# Error Model
For the error model: http://norvig.com/ngrams/count_1edit.txt

This file contains single-edit spelling correction edits and is based on Peter Norvig's collection of spelling mistakes. The data originates from Wikipedia. The actual words and their misspelling can be found at https://norvig.com/ngrams/spell-errors.txt

The file is structured in the following manner: w|c z ; where w is the wrong character, c is the correct, and z is the occurrence of the error

For example, in count_1edit.txt, there is 1 occurrence of B|M, where B was substituted M in the example My: By (here the correct word is My but it was misspelled as By).

You can normalize the counts based on your language model (i.e. count edit sequences in the corpus)

Note: defaultdict subclass was used to count errors without needing to manually check if a key already exists in the dictionary
https://www.geeksforgeeks.org/defaultdict-in-python/

In [3]:
filepath = "count_1edit.txt"

def load_error_model(filepath):
    error_model = defaultdict(int)
    with open(filepath, 'r') as file:
        for line in file:
            match = re.match(r"(\w)\|(\w)\s(\d+)", line) # regex to extract errors and their count
            if match:
                wrong_char, correct_char, count = match.groups()
                error_model[(wrong_char, correct_char)] += int(count)
    return error_model

error_model = load_error_model(filepath)

# display 10 lines from the error model
print("Error Model Sample:")
for i, j in list(error_model.items())[:10]:
    print(f"Error: {i}, Count: {j}")

Error Model Sample:
Error: ('e', 'i'), Count: 917
Error: ('a', 'e'), Count: 856
Error: ('i', 'e'), Count: 771
Error: ('e', 'a'), Count: 749
Error: ('a', 'i'), Count: 559
Error: ('s', 'c'), Count: 383
Error: ('a', 'o'), Count: 353
Error: ('o', 'a'), Count: 352
Error: ('i', 'a'), Count: 313
Error: ('e', 'o'), Count: 295


# Language Model
For the language model: NLTK's Gutenberg Corpus
Information: https://www.nltk.org/book/ch02.htmlLinks

This is a walk-through of using and collating all documents within the collection: Using_All_Project_Gutenberg_Documents_in_NLTK.ipynb

The walk-through also gives some brief statistics of the corpus

Note: The first 3 code blocks below are from the provided walk-through of Gutenberg's documents.

In [4]:
nltk.download('gutenberg')
nltk.corpus.gutenberg.fileids()

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [5]:
print("Extracting all documents from NLTK's Project Gutenberg Collection...")
all_words = Counter()
for filename in nltk.corpus.gutenberg.fileids():
  words = [word.lower() for word in nltk.corpus.gutenberg.words(filename)]
  all_words.update(words)
  print("%s; tokens: %d; vocab: %d" % (filename, len(words), len(set(words))))

Extracting all documents from NLTK's Project Gutenberg Collection...
austen-emma.txt; tokens: 192427; vocab: 7344
austen-persuasion.txt; tokens: 98171; vocab: 5835
austen-sense.txt; tokens: 141576; vocab: 6403
bible-kjv.txt; tokens: 1010654; vocab: 12767
blake-poems.txt; tokens: 8354; vocab: 1535
bryant-stories.txt; tokens: 55563; vocab: 3940
burgess-busterbrown.txt; tokens: 18963; vocab: 1559
carroll-alice.txt; tokens: 34110; vocab: 2636
chesterton-ball.txt; tokens: 96996; vocab: 8335
chesterton-brown.txt; tokens: 86063; vocab: 7794
chesterton-thursday.txt; tokens: 69213; vocab: 6349
edgeworth-parents.txt; tokens: 210663; vocab: 8447
melville-moby_dick.txt; tokens: 260819; vocab: 17231
milton-paradise.txt; tokens: 96825; vocab: 9021
shakespeare-caesar.txt; tokens: 25833; vocab: 3032
shakespeare-hamlet.txt; tokens: 37360; vocab: 4716
shakespeare-macbeth.txt; tokens: 23140; vocab: 3464
whitman-leaves.txt; tokens: 154883; vocab: 12452


In [8]:
print("Overall Statistics")
total_tokens = sum(all_words.values())
total_types = len(all_words)

print("Total tokens: %d" % total_tokens)
print("Total vocabulary: %d" % total_types)
print("Type/token ratio: %.4f" % (total_types / total_tokens))
print("Vocabulary richness: %.4f" % (total_types / (total_tokens ** (1/2))))

Overall Statistics
Total tokens: 2621613
Total vocabulary: 42339
Type/token ratio: 0.0161
Vocabulary richness: 26.1491


Language Model Formula:
count of word (w) / Total tokens

In [7]:
language_model = {word: count / total_tokens for word, count in all_words.items()}

# print results for observation
print("\nLanguage Model (word probabilities):")
for word, prob in list(language_model.items())[10:20]:
    print(f"{word}: {prob:.8f}")


Language Model (word probabilities):
woodhouse: 0.00011939
,: 0.07098340
handsome: 0.00005035
clever: 0.00002975
and: 0.03640583
rich: 0.00009155
with: 0.00671304
a: 0.01295386
comfortable: 0.00004120
home: 0.00025976


#Preparing the Noisy Channel Model
The goal is to implement a model based on probabilities of spelling errors.

This is done by generating all 1 edit words from the input word and then query the language model for the most probable word.

The vocabulary is limited to 1-grams and candidate words with edit distance of 1 with respect to the input word.

NOTE: Error Correction only happens if an error is detected (if word does NOT exist in the Language Model)

References:
* https://norvig.com/spell-correct.html?fbclid=IwZXh0bgNhZW0CMTEAAR2qMQfJOaK2iDTeJzxv9gIWZKVclmVPdnKAXTr4Up6MV1pvrjz9MgqZVX4_aem_O8k85dDmd9xdgOtKfvEj9g
* https://www.geeksforgeeks.org/defaultdict-in-python/

Check if word exists in the Language Model.
If it exists, no need to perform Spell Correction

In [9]:
def is_word_correct(word):
    return word in language_model

is_word_correct('ass')

True

Get all candidate words with Edit Distance of 1

In [10]:
# Peter Norvig's Spelling Corrector
def edits1(word):
    # Generate all words with one edit distance from the given word
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [(L + R[1:], "del", f"{R[0]}|-") for L, R in splits if R]
    transposes = [(L + R[1] + R[0] + R[2:], "tra", f"{R[0]+R[1]}|{R[1]+R[0]}") for L, R in splits if len(R) > 1]
    substitutes = [(L + c + R[1:], "sub", f"{R[0]}|{c}") for L, R in splits if R for c in letters]
    inserts = [(L + c + R, "ins", f"-|{c}") for L, R in splits for c in letters]

    return set(deletes + transposes + substitutes + inserts)

Computing Error Probability for Candidate Words

In [11]:
def error_probability(wrong, correct):
    # Get Error probability P(w|c) from Error model (defaultdict)
    wrong_char = wrong[0]
    correct_char = correct[0]
    return error_model.get((wrong_char, correct_char), 1) / sum(error_model.values()) # returns probability of
    # Note: 1 is used as a default value in the case that the error isn't recorded in the error model

#Spell Correction (Noisy Channel Model)

In [12]:
def spell_correct(word):
    if is_word_correct(word):
        print(f"No error: {word} is correctly spelled.")
        return [(word, "None", "-", language_model[word], "-", "-")]

    candidates = edits1(word) # generate candidates with edit distance of 1
    final_candidates = [] # initialize a list to store final candidates

    for candidate, edit_type, edit in candidates: # loops for each candidate
        if candidate in language_model: # double check to see if the generated candidate word is included in our vocab
            prob_c = language_model[candidate]  # P(c)
            prob_w_given_c = error_probability(word, candidate)  # P(w|c)
            final_prob = prob_c * prob_w_given_c
            final_candidates.append((candidate, edit_type, edit, prob_c, prob_w_given_c, final_prob)) # add the results for each candidate to the list

    final_candidates.sort(key=itemgetter(5), reverse=True) # sort the final candidates by P(w) * P(w|c)
    return final_candidates


#Testing the Spell Corrector

In [14]:
word = "mother"  # misspelled word
print("\nCorrection Candidates for:", word)
candidates = spell_correct(word)

print(f"\n{'word':>20}\t{'candidate':>20}\tedit_type\t{'edit':>30}\t{'P(c)':>12}\t{'P(w|c)':>12}\t{'P(c) x P(w|c)':>15}")
for candidate in candidates:
    if type(candidate[3]) is str:
        format3 = "{:>12}".format(" - ")
    else:
        format3 = "{:>12.6f}".format(candidate[3])

    if type(candidate[4]) is str:
        format4 = "{:>12}".format(" - ")
    else:
        format4 = "{:>12.6f}".format(candidate[4])

    if type(candidate[5]) is str:
        format5 = "{:>15}".format(" - ")
    else:
        format5 = "{:>15.6e}".format(candidate[5])

    print(f"{word:>20}\t{candidate[0]:>20}\t{candidate[1]:>9}\t{candidate[2]:>30}\t{format3}\t{format4}\t{format5}")


Correction Candidates for: mother
No error: mother is correctly spelled.

                word	           candidate	edit_type	                          edit	        P(c)	      P(w|c)	  P(c) x P(w|c)
              mother	              mother	     None	                             -	    0.000463	          - 	             - 


#Other References
The following code (Damerau-Levenshtein algorithm) was used as a reference for an earlier implementation, before the group decided to use Norvig's Spelling Corrector.

https://stackoverflow.com/questions/44640570/modify-damerau-levenshtein-algorithm-to-track-transformations-insertions-delet?fbclid=IwZXh0bgNhZW0CMTEAAR3flr2AriGp4RHGaswU2hfeen0gyzxB2xWRGbbqN2P1vkdupPSEpQ3ctvc_aem_XImT6tAKsEwjHxLYYXA4pg

In [ ]:
def levenshtein_distance(string1, string2):
    n1 = len(string1)
    n2 = len(string2)
    return _levenshtein_distance_matrix(string1, string2)[n1, n2]

def damerau_levenshtein_distance(string1, string2):
    n1 = len(string1)
    n2 = len(string2)
    return _levenshtein_distance_matrix(string1, string2, True)[n1, n2]

def get_ops(string1, string2, is_damerau=False):
    i, j = _levenshtein_distance_matrix(string1, string2, is_damerau).shape
    i -= 1
    j -= 1
    ops = list()
    while i != -1 and j != -1:
        if is_damerau:
            if i > 1 and j > 1 and string1[i-1] == string2[j-2] and string1[i-2] == string2[j-1]:
                if dist_matrix[i-2, j-2] < dist_matrix[i, j]:
                    ops.insert(0, ('transpose', i - 1, i - 2))
                    i -= 2
                    j -= 2
                    continue
        index = np.argmin([dist_matrix[i-1, j-1], dist_matrix[i, j-1], dist_matrix[i-1, j]])
        if index == 0:
            if dist_matrix[i, j] > dist_matrix[i-1, j-1]:
                ops.insert(0, ('replace', i - 1, j - 1))
            i -= 1
            j -= 1
        elif index == 1:
            ops.insert(0, ('insert', i - 1, j - 1))
            j -= 1
        elif index == 2:
            ops.insert(0, ('delete', i - 1, i - 1))
            i -= 1
    return ops

def execute_ops(ops, string1, string2):
    strings = [string1]
    string = list(string1)
    shift = 0
    for op in ops:
        i, j = op[1], op[2]
        if op[0] == 'delete':
            del string[i + shift]
            shift -= 1
        elif op[0] == 'insert':
            string.insert(i + shift + 1, string2[j])
            shift += 1
        elif op[0] == 'replace':
            string[i + shift] = string2[j]
        elif op[0] == 'transpose':
            string[i + shift], string[j + shift] = string[j + shift], string[i + shift]
        strings.append(''.join(string))
    return strings

def _levenshtein_distance_matrix(string1, string2, is_damerau=False):
    n1 = len(string1)
    n2 = len(string2)
    d = np.zeros((n1 + 1, n2 + 1), dtype=int)
    for i in range(n1 + 1):
        d[i, 0] = i
    for j in range(n2 + 1):
        d[0, j] = j
    for i in range(n1):
        for j in range(n2):
            if string1[i] == string2[j]:
                cost = 0
            else:
                cost = 1
            d[i+1, j+1] = min(d[i, j+1] + 1, # insert
                              d[i+1, j] + 1, # delete
                              d[i, j] + cost) # replace
            if is_damerau:
                if i > 0 and j > 0 and string1[i] == string2[j-1] and string1[i-1] == string2[j]:
                    d[i+1, j+1] = min(d[i+1, j+1], d[i-1, j-1] + cost) # transpose
    return d

if __name__ == "__main__":
    # GIFTS PROFIT
    # FBBDE BCDASD
    # SPARTAN PART
    # PLASMA ALTRUISM
    # REPUBLICAN DEMOCRAT
    # PLASMA PLASMA
    # FISH IFSH
    # STAES STATES
    string1 = 'MOTHER'
    string2 = 'MOHTER'
    for is_damerau in [True, False]:
        if is_damerau:
            print('=== damerau_levenshtein_distance ===')
        else:
            print('=== levenshtein_distance ===')
        dist_matrix = _levenshtein_distance_matrix(string1, string2, is_damerau=is_damerau)
        print(dist_matrix)
        ops = get_ops(string1, string2, is_damerau=is_damerau)
        print(ops)
        res = execute_ops(ops, string1, string2)
        print(res)

=== damerau_levenshtein_distance ===
[[0 1 2 3 4 5 6]
 [1 0 1 2 3 4 5]
 [2 1 0 1 2 3 4]
 [3 2 1 1 1 2 3]
 [4 3 2 1 1 2 3]
 [5 4 3 2 2 1 2]
 [6 5 4 3 3 2 1]]
[('transpose', 3, 2)]
['MOTHER', 'MOHTER']
=== levenshtein_distance ===
[[0 1 2 3 4 5 6]
 [1 0 1 2 3 4 5]
 [2 1 0 1 2 3 4]
 [3 2 1 1 1 2 3]
 [4 3 2 1 2 2 3]
 [5 4 3 2 2 2 3]
 [6 5 4 3 3 3 2]]
[('replace', 2, 2), ('replace', 3, 3)]
['MOTHER', 'MOHHER', 'MOHTER']
